In [1]:
from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext
import sys
import requests
import time
import subprocess
import re
from google.cloud import bigquery
import datetime
from pyspark.sql.functions import lit, unix_timestamp


In [2]:

bucket = "yuqintwitter"   
output_directory_hashtags = 'gs://{}/hadoop/tmp/bigquery/pyspark_output/hashtagsCount'.format(bucket)
output_directory_wordcount = 'gs://{}/hadoop/tmp/bigquery/pyspark_output/wordcount'.format(bucket)

# output table and columns name
output_dataset = 'twitter'               
output_table_hashtags = 'hashtags'
columns_name_hashtags = ['hashtags', 'count']
output_table_wordcount = 'wordcount'
columns_name_wordcount = ['word', 'count', 'time']

# parameter
IP = 'localhost'    # ip port
PORT = 9001       # port

STREAMTIME = 600          # time that the streaming process runs

WORD = ['data', 'spark', 'ai', 'movie', 'good']  

timemark=datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')

In [3]:

def saveToBigQuery(sc, output_dataset, output_table, directory):
    files = directory + '/part-*'
    subprocess.check_call(
        'bq load --source_format NEWLINE_DELIMITED_JSON '
        '--replace '
        '--autodetect '
        '{dataset}.{table} {files}'.format(
            dataset=output_dataset, table=output_table, files=files
        ).split())
    output_path = sc._jvm.org.apache.hadoop.fs.Path(directory)
    output_path.getFileSystem(sc._jsc.hadoopConfiguration()).delete(
        output_path, True)

In [4]:
def regular_expression(word):
    if re.match('^#[a-zA-Z0-9]+',word):
        return True
    else:
        return False
def update_state(value,historyvalue):
    if historyvalue is None:
        historyvalue=0
    return sum(value,historyvalue)
    
def hashtagCount(words):
    hashword1=words.map(lambda word:word.lower()).filter(regular_expression).map(lambda word:(word,1))
    hashword=hashword1.updateStateByKey(update_state).transform(lambda wordmap:wordmap.sortBy(lambda word:word[1],ascending=False))
    return hashword
    

In [5]:
def wordCount(words):
    counting=words.map(lambda w:w.lower()).filter(lambda w:w in WORD).map(lambda w:(w,1)) \
    .reduceByKeyAndWindow(lambda a,b:a+b,lambda a,b:a-b,60,60).transform(lambda timemark,count:count.map(lambda c:(c[0],c[1],timemark)))
    return counting
                                                                                                
    

In [6]:
def saveToStorage_H(rdd):
    if not rdd.isEmpty():
        rdd.toDF( columns_name_hashtags ).orderBy('count', ascending=False).write.save(output_directory_hashtags, format="json", mode="overwrite")

def saveToStorage_W(rdd):
    if not rdd.isEmpty():
        rdd.toDF( columns_name_wordcount ).orderBy(['word']).write.save(output_directory_wordcount, format="json", mode="append")

In [ ]:
if __name__ == '__main__':
    # Spark settings
    conf = SparkConf()
    conf.setMaster('local[2]')
    conf.setAppName("TwitterStreamApp")

    # create spark context with the above configuration
    sc.stop()
    sc = SparkContext(conf=conf)
    sc.setLogLevel("ERROR")

    # create sql context, used for saving rdd
    sql_context = SQLContext(sc)

    # create the Streaming Context from the above spark context with batch interval size 5 seconds
    ssc = StreamingContext(sc, 5)
    # setting a checkpoint to allow RDD recovery
    ssc.checkpoint("~/checkpoint_TwitterApp")

    # read data from port 9001
    dataStream = ssc.socketTextStream(IP, PORT)
    dataStream.pprint()

    words = dataStream.flatMap(lambda line: line.split(" "))

    # calculate the accumulated hashtags count sum from the beginning of the stream
    topTags = hashtagCount(words)
    topTags.pprint()

    # Calculte the word count during each time period 6s
    wordCount = wordCount(words)
    wordCount.pprint()

    topTags.foreachRDD(saveToStorage_H)
    wordCount.foreachRDD(saveToStorage_W)
    # start streaming process, wait for 600s and then stop.
    ssc.start()
    time.sleep(STREAMTIME)
    ssc.stop(stopSparkContext=False, stopGraceFully=True)

    # put the temp result in google storage to google BigQuery
    saveToBigQuery(sc, output_dataset, output_table_hashtags, output_directory_hashtags)
    saveToBigQuery(sc, output_dataset, output_table_wordcount, output_directory_wordcount)


-------------------------------------------
Time: 2021-10-20 23:35:00
-------------------------------------------

-------------------------------------------
Time: 2021-10-20 23:35:00
-------------------------------------------

-------------------------------------------
Time: 2021-10-20 23:35:05
-------------------------------------------
RT @Grindpod: The boys are joined by @PerryVonVicious to discuss a movie about love, loss, and a murderous monster truck with a giant drill…Another movie clip sold via #Adultwork.com! https://t.co/M5p0OI9y45 HARDCORE - Jessica Bangkok Sucks and FucksRT @vvs_films: #AfterWeFell is now the #1 movie on @AppleTV! Watch it today everywhere you purchase movies online! Check out the link in ou…RT @jimmy_dmg: quand j’étais en seconde j’avais triché à un contrôle et j’avais eu 20/20 la prof elle a remarqué que j’avais tricher elle m…@Post_Market Correct - not a recurring business.  We will know the answer to how good the UPST “AI” underwriting is… https://t

-------------------------------------------
Time: 2021-10-20 23:35:25
-------------------------------------------
#ranboofanart #ranboo #generationloss https://t.co/yH9vsa5IkfRT @jaythechou: I Photoshop Paddington into another movie until I forget: Day 224 https://t.co/M9cKEXTg97MULTI ADSBX MANOEUVRING ALERT : At time Thu Oct 21 00:19:51 2021 # was likely to be manoeuvring at FL74 2nm from RI… https://t.co/sdVf18C5dI@ilycloures @tega_life 😭😤🤡😌👍🏽🙇🏽‍♀️ KKKKKK aiRT @kelyagl: La vie d'adulte est trop difficile frère donc si mes ami.e.s ne me demandent pas si ça va, je ne leur en veux même pas. Aussi,…@McSkyzyoutube J'ai vu cette affaire c'est vraiment triste 😭😭😭@icedwaterpapi not you sounding excited for this movieRT @chifuyussmile: # BUSCO MUTUALS.

no soy nuevo en #anitwt #animetwt, pero volví a activar y quiero mutuals. 

tr, haikyuu, bsd, jjk, kny…https://t.co/8aS0xMKz5NRT @percymadraw: Un certain nombre d’heures et une cinquantaine de crayons plus tard… j’ai finis ma réalisation de @B

-------------------------------------------
Time: 2021-10-20 23:35:40
-------------------------------------------
('#nft', 6)
('#ai', 6)
('#shehulk', 5)
('#art', 5)
('#cryptodragons', 5)
('#hulk', 4)
('#worldwarhulk', 4)
('#machinelearning', 4)
('#squidgames', 4)
('#marvelstudios', 4)
...

-------------------------------------------
Time: 2021-10-20 23:35:45
-------------------------------------------
https://…@Jollyro97060420 ai eu vou ficar devendoRT @Catona_97: des fois je m’ambiance tellement dans la voiture que j’ai le seum d’être arrivéhttps://t.co/sdady5VGJ1RT @allkpop: [UCC] BTS' Jimin's self-written, composed, and produced song 'Friends' is played during an entire scene in the upcoming Marvel…Vou beber plena quarta feira e to nem ai to triste https://t.co/O6lS2hDxYnparfois j’ai envie d’faire des trucs de boloss mais heureusement j’me ressaisis vite 🫂RT @fratotolo2: Nel completo silenzio mediatico e politico, a ottobre sono già sbarcati in Italia 4.211 #migranti, 50.540 dall'in

-------------------------------------------
Time: 2021-10-20 23:36:00
-------------------------------------------
('#ai', 8)
('#nft', 7)
('#shehulk', 6)
('#art', 6)
('#squidgames', 6)
('#cryptodragons', 6)
('#hulk', 5)
('#rt', 5)
('#machinelearning', 5)
('#marvelstudios', 5)
...

-------------------------------------------
Time: 2021-10-20 23:36:05
-------------------------------------------
Meu pai quase atropelou esse dog indo viajar , ai pra não deixar o cachorro obviamente em risco,… https://t.co/9rQhp8zljSRT @Mouten____: https://t.co/GV93bSd5MjRT @PSGalu: This guy is living a movie man, no way his career isn't scripted, what the hell Ronaldo@PLocomodor Aqui amg, o tutorial de como fazer, essas configurações todas ai vc so vai fazer pra habilitar, nas pró… https://t.co/9ksAHKCedTRT @Boydamvn: Nói về độ dâm thì anh này cũng khá đỉnh. Còn nhiều clip khẩu dâm cực dâm của ảnh. Ai muốn xem không nào https://t.co/BkKv3dyu…RT @1st_Army_Area: # จริงจัง ต่อเนื่อง บริการด้วยหัวใจ #
 
กองพันซ

-------------------------------------------
Time: 2021-10-20 23:36:30
-------------------------------------------
@tomskiweb @reubenyap @firoorg #BTC $BTC #Bitcoin #bullish #breakout #altco…Eu tava falando que o Abel era burro de colocar ele, ai ele vai lá e faz um golRT @LucaBizzarri: Se non fossero violenti, sarebbero solo degli imbecilli.$xyo is the way to go 😂🚀🤑🤑@frdossant @TristanKamin @Down_Ra226 @BzhClair @KarlSpookbuster T'as vu Karl, j'ai écrit tripes, pas couilles. Devi… https://t.co/n3Z7bcAshhCan’t wait to see what’s going on with https://t.co/fCSpkgWKd1 on Nov. 18th? Are we getting a reunion or a new movi… https://t.co/JAPEu8dYwS@ifsmiley eu tô namorando um italiano se fode aiRT @oxlxlo: 이거 나왔다고 웹작 날로먹는다는등의 쌉소리하면 타블렛펜으로 찔러버린다 진짜FRUTABLEND # KULIT CANTIK DALAM HITUNGAN HARI+BONUS BADAN SEHAT. ORDR bb27df4e5c/ 082218808138 https://t.co/2YKsKSjPtlCódigo Prime do @BRGenshinImpact ai pra vcs: TSPWYZY5HEEZ ! https://t.co/ERh2CcfTtCRT @Rifton_Salah: Le plus que j’ai vu c’était ma 

...

-------------------------------------------
Time: 2021-10-20 23:42:10
-------------------------------------------
텔 @ zz944 ##실계정RT @nUKiOz: สหภาพแรงงานรวมตัวประท้วงในเกาหลีใต้ แต่งตัวคอสตูม #SquidGames .. ระบุว่า ชีวิตพวกเค้าก็ไม่ต่างจากในซีรีส์ที่ต้องดิ้นรนหาเลี้ยง…Information Theory: Principles and Apostasy https://t.co/s1tjrZC3Fk #WeWantData #Data #Insights #Statistics #AI… https://t.co/hHvNyWBOuKC’était quand j’étais encore sous mon ancien compte @blazerpurpose 😭😭Pronto agora vi o pau do thanos ai q desgraçaRT @mailyys___: Ça tombe bien j’ai une ptite question au sujet de ton TN https://t.co/M0xSava1McRT @hrrykarma: sentar perto de um nerdola no cinema só pra falar quem é esse thanos ai mesmoUpon further review….Waiting to Exhale just off the strength of “Not Gon Cry” -Mary J Blige…..CHEAT CODERT @SkipFromBk: 👀👀👀👀👀👀👀👀👀👀👀RT @Seba_Terry: I was watching a movie with my straight friend and this happened… 🎞️🔥
Just for this month 🔥60%📴at https://t.co/6d6tK3cCXh o…RT @yunsukCNA: Several 

-------------------------------------------
Time: 2021-10-20 23:42:35
-------------------------------------------
htt…Je suis grosse, jsuis archi pas contre le régime, mais tant que je n'ai pas de chez moi c'est compliqué d'avoir un… https://t.co/cuz9aseB9YExatamente!!ai caro ter um kindle ne mas queria 🥲RT @MaryLTrump: The @MeidasTouch strikes again!

This is going to be amazing. 👇🏼RT @5ecretNumber: “Fire Saturday” SECRET Spoiler # 05

SECRET NUMBER
3rd Single Album [Fire Saturday]
2021.10.27 6PM(KST) Release 

https:/…RT @jackiemorelli: #womenintech #django #nocode #javascript #github #nodejs #cybersecurity #devops #100DaysOfCode #datascience #ai #bot #co…Gabriel, você tá ouvindo Andrew Belle, tô chocado, mas qual a sua fav e pq é honey and milk? 😱😱😱😱😱😱 — kkk pse, eu a… https://t.co/gQlf6ktyP7@nathandomdq @bffdonico @Melancia ai nathan apaga isso pro meu bemRT @Charan151515: Bahubali Rana role ki RRR multistarrer ki.. RC15 Shankar movie ki Ghazi movie ki comparison Entraa https://t.c

-------------------------------------------
Time: 2021-10-20 23:42:55
-------------------------------------------
('movie', 179, datetime.datetime(2021, 10, 20, 23, 42, 55))
('ai', 294, datetime.datetime(2021, 10, 20, 23, 42, 55))
('good', 10, datetime.datetime(2021, 10, 20, 23, 42, 55))
('spark', 6, datetime.datetime(2021, 10, 20, 23, 42, 55))
('data', 4, datetime.datetime(2021, 10, 20, 23, 42, 55))

-------------------------------------------
Time: 2021-10-20 23:43:00
-------------------------------------------
2020                                       20…https://t.co/C33xNn9cDf@Rodrigo29000438 @spfcpics Ai no basquete eu concordo em ter todo mundo, pq fizeram um puta projetoRT @lobotosasha: Propaganda teen de uma doença horrível KKKKKKKKKKK

Aí ai esse marketing brazuca https://t.co/ytwjl8BUsgWhen u know, u know it. HAHAHHAAHAHAje sens comme si j’ai pas d’avenir c’est trop énervantRT @latestmermaid: bora anavitória lança um álbum ai tambémI described what being shomer Shabbat meant

-------------------------------------------
Time: 2021-10-20 23:43:30
-------------------------------------------
https://t.co/nJNezWP5HN https://t.co/mYvJq6dhDPRT @tamoioart: com esse video rolando ai pela net eu to só nesse meme https://t.co/J3oWLkEgSTRT @sunya_XD: 네이버측에서 이 프로그램이 사용된 그림 저장해둔다네요 머 무단전재를 하겠느냐만 찜찜하니까 중요한 작업물은 돌리지마시구 심심풀이용이나 작은 덕질에 쓰시기RT @pinto_emelo: Moi mettre encore un pied dans une église? Dans vos rêves.  je prie chez moi. J’ai absolument 0 confiance en eux.minha segunda dose mes q vem ai papaiRT @StrasserCaio: quem ta me devendo ai, faz o pix, quero ir na monkey 🤣RT @NerdKid_94: Cried like a bitch sampai kena minum sebab takut mati dehydrated.

10/10

"Violet Evergarden the Movie" on Netflix yet?

ht…fui comprar meus tênis e não tinha mais, ai o carinha salvou meu número e eu o dele p quando chegasse ele me avisar… https://t.co/BrOX1VD78YRT @islandtv_up: 菅田、安嶋、稲葉、川﨑星輝、長瀬、林、松尾「みっちー生誕祭！」
#菅田琳寧
#7MEN侍
#安嶋秀生 #稲葉通陽 #川﨑星輝 #長瀬結星
...

--------------------------------------

-------------------------------------------
Time: 2021-10-20 23:44:40
-------------------------------------------

-------------------------------------------
Time: 2021-10-20 23:44:40
-------------------------------------------
('#ai', 111)
('#squidgames', 51)
('#nft', 44)
('#art', 39)
('#100daysofcode', 39)
('#hulk', 37)
('#python', 37)
('#marvelstudios', 36)
('#cryptodragons', 35)
('#machinelearning', 34)
...

-------------------------------------------
Time: 2021-10-20 23:44:45
-------------------------------------------

-------------------------------------------
Time: 2021-10-20 23:44:45
-------------------------------------------
('#ai', 111)
('#squidgames', 51)
('#nft', 44)
('#art', 39)
('#100daysofcode', 39)
('#hulk', 37)
('#python', 37)
('#marvelstudios', 36)
('#cryptodragons', 35)
('#machinelearning', 34)
...

-------------------------------------------
Time: 2021-10-20 23:44:50
-------------------------------------------

-------------------------------------------
Time: 2